# Importamos librerías y conectamos al drive

In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.tokenize import word_tokenize 
from   nltk.tokenize import TreebankWordTokenizer 
from   nltk.stem     import SnowballStemmer, WordNetLemmatizer #PorterStemmer
from   nltk.corpus   import stopwords #lista de stop words

nltk.download('wordnet') #me bajo los datos para poder usarlos
nltk.download('punkt')
nltk.download('stopwords')

tokenizer  = TreebankWordTokenizer()
stemmer    = SnowballStemmer("english") #PorterStemmer
lemmatizer = WordNetLemmatizer()

 #Importo los vectorizadores
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import pickle #para guardado de datos
from sklearn.naive_bayes             import MultinomialNB
import os
#!pip install varname
#from varname import nameof #para obtener el nombre de variables

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
%cd "/content/drive/Shareddrives/Redes neuronales/TP1"

/content/drive/Shareddrives/Redes neuronales/TP1


In [44]:
!pip install import-ipynb

In [45]:
import import_ipynb
from Helper import *

# Cargo los datos

In [46]:
df_train = pd.read_hdf("/content/drive/Shareddrives/Redes neuronales/TP1/data/train_data.hdf5")
df_valid = pd.read_hdf("/content/drive/Shareddrives/Redes neuronales/TP1/data/valid_data.hdf5")
df_test = pd.read_hdf("/content/drive/Shareddrives/Redes neuronales/TP1/data/test_data.hdf5")

In [47]:
#Cantidad de documentos en train
len(df_train)

549367

In [48]:
df_train.head()

,gold_label,text
pairID,,
1000092795.jpg#0r1c,contradiction,they are inside of a house
1000092795.jpg#0r1e,entailment,two guys are in a yard
1000092795.jpg#0r1n,neutral,They are doing yardwork
1000092795.jpg#1r1c,contradiction,A man is swimming.
1000092795.jpg#1r1e,entailment,Two young white men are near some bushes.


In [49]:
df_train['text'][0]

'they are inside of a house'

In [50]:
df_train['gold_label'].values

array(['contradiction', 'entailment', 'neutral', ..., 'contradiction',
       'entailment', 'neutral'], dtype=object)

In [51]:
df_valid.head()

,gold_label,text
pairID,,
100197432.jpg#4r1c,contradiction,The women are seated in a bistro eating lunch.
100197432.jpg#4r1e,entailment,There are muliple women outdoors.
100197432.jpg#4r1n,neutral,The women are friends.
1026792563.jpg#1r1c,contradiction,A group of women are fishing
1026792563.jpg#1r1e,entailment,A group of women are selling their wares


In [52]:
df_test.head()

,text
pairID,
0,The church has cracks in the ceiling.
1,The church is filled with song.
2,A choir singing at a baseball game.
3,The woman is young.
4,The woman is very happy.


In [53]:
 #df_submission = pd.read_csv("/kaggle/input/sesgos-en-el-dataset-de-snli/submission_sample.csv", index_col="pairID")


In [54]:
#df_submission

In [55]:
text_train = df_train["text"].tolist() #nos quedamos con los textos
labels_train = df_train["gold_label"].tolist() #nos quedamos con la clasificación
text_val = df_valid["text"].tolist()
labels_val = df_valid["gold_label"].tolist()
text_test = df_test["text"].tolist()

In [56]:
#Veamos el balance de clases
Counter(labels_train)
#esta bastante parejo, dataset equilibrado

Counter({'contradiction': 183187, 'entailment': 183416, 'neutral': 182764})

In [57]:
len(labels_val)

9842

In [58]:
len(text_val)

9842

##Mezclo los datos de train y valid para hacer luego cross validation

In [59]:
df_train_valid = pd.concat([df_train,df_valid])
len(df_train_valid)

559209

In [60]:
len(df_train) + len(df_valid)

559209

In [61]:
text_train_valid = df_train_valid["text"].tolist() #nos quedamos con los textos
labels_train_valid = df_train_valid["gold_label"].tolist() #nos quedamos con la clasificación

# Elección de parámetros de preprocesamiento e hiperparámetros

Stemming también se llama estandarización de texto donde las palabras se reducen a su forma base/raíz. Por ejemplo, palabras como "programador", "programación," programa "se derivarán de" programa ".

La desventaja de stemming es que reduce a las palabreas de tal manera que su forma raíz pierde el significado, se puede reducit a una forma que deja de ser una palabras en inglés adecuada. Ejemplo: crazy -> crazi, early -> earli, available -> avail.

En cambio, lemmatize se reduce la palabra a su raíz pero se asegura de que no pierda su significado. La lemmatización tiene un diccionario predefinido que almacena el contexto de las palabras y verifica que la palabra esté en el diccionario mientras la reduce.

##Creación de combinaciones de parámetros de procesamiento e hiperparámetros

In [62]:
#definición de parámetros de preprocesamiento
parameters_specs = {  
    'is_lc': [True, False],
    'is_lem': [True, False],
    'is_stop': [True, False],
    'is_stem': [True, False],
    'is_alpha': [True, False],
    'alpha':   [1e-10,1e-5,0.01,1,2],#0.01, 0.1, 1.0,10.0 #smooothing laplaciano (para sumar cuando frec=0), más grande, más parecido al uniforme
    'min_df':  [0.1,0.0001,0.00001,0.000002], #0.01, 0.1, 0.05, 0.49, 1, 10 #porcentaje mínimo de aparición en documentos
    'max_df':  [0.4, 0.8,0.9], # 0.5, 0.7,#porcentaje máximo de aparición en documentos
    'tf_idf':  [True, False], #usar tfidf o countVectorizer
}

In [63]:
#hacemos las combinaciones
def get_parameters():
      for is_lc in parameters_specs['is_lc']:
          for is_lem in parameters_specs['is_lem']:
              for is_stop in parameters_specs['is_stop']:
                  for is_stem in parameters_specs['is_stem']:
                    for is_alpha in parameters_specs['is_alpha']:
                      for tf_idf in parameters_specs['tf_idf']:
                        for min_df in parameters_specs['min_df']:
                            for max_df in parameters_specs['max_df']:
                                for alpha in parameters_specs['alpha']:
                                    parameters = {
                                        'is_lc': is_lc,
                                        'is_lem': is_lem,
                                        'is_stop': is_stop,
                                        'is_stem': is_stem,
                                        'is_alpha': is_alpha,
                                        'alpha':   alpha,
                                        'min_df':  min_df,
                                        'max_df':  max_df,
                                        'tf_idf':  tf_idf
                                    }
                      
                                    yield parameters #gives a generator function, para evitar muchos llamados

In [64]:
preproc_hyperparams = pd.DataFrame()

for p in get_parameters():
    df = pd.DataFrame(p, index=[0])
    preproc_hyperparams = preproc_hyperparams.append(df,ignore_index=True) #ponemos combinaciones

preproc_hyperparams 

,is_lc,is_lem,is_stop,is_stem,is_alpha,alpha,min_df,max_df,tf_idf
0,True,True,True,True,True,1.000000e-10,0.100000,0.4,True
1,True,True,True,True,True,1.000000e-05,0.100000,0.4,True
2,True,True,True,True,True,1.000000e-02,0.100000,0.4,True
3,True,True,True,True,True,1.000000e+00,0.100000,0.4,True
4,True,True,True,True,True,2.000000e+00,0.100000,0.4,True
...,...,...,...,...,...,...,...,...,...
3835,False,False,False,False,False,1.000000e-10,0.000002,0.9,False
3836,False,False,False,False,False,1.000000e-05,0.000002,0.9,False
3837,False,False,False,False,False,1.000000e-02,0.000002,0.9,False
3838,False,False,False,False,False,1.000000e+00,0.000002,0.9,False


In [65]:
len(preproc_hyperparams)

3840

## Vemos los resultados variando los parámetros entrenando con 100.000 hipotesis del conjunto de train

In [66]:
len(text_train_valid)

559209

In [67]:
Counter(labels_train[0:200000])

Counter({'contradiction': 66691, 'entailment': 66764, 'neutral': 66545})

In [68]:
preproc_hyperparams_70 = preproc_hyperparams.sample(frac=0.1)
preproc_hyperparams_70 #0.7

,is_lc,is_lem,is_stop,is_stem,is_alpha,alpha,min_df,max_df,tf_idf
3733,False,False,False,False,False,1.000000e+00,0.100000,0.9,True
2619,False,True,False,True,False,2.000000e+00,0.000010,0.8,False
577,True,True,False,True,True,1.000000e-02,0.000010,0.8,False
1332,True,False,True,False,False,1.000000e-02,0.100000,0.9,True
3116,False,False,True,True,False,1.000000e-05,0.000002,0.9,False
...,...,...,...,...,...,...,...,...,...
2124,False,True,True,True,False,2.000000e+00,0.000100,0.8,False
1243,True,False,True,False,True,1.000000e+00,0.000010,0.9,True
397,True,True,True,False,False,1.000000e-02,0.000010,0.8,True
3705,False,False,False,False,True,1.000000e-10,0.000002,0.4,False


In [ ]:
#tardo 38 min 20s
scores = []
for index, p in preproc_hyperparams_70.iterrows():
    print(p.to_dict())
    score = crossval_score_parameters_preprocessing_loaded(text_train_valid[0:300000],labels_train_valid[0:300000],is_lc_=p['is_lc'],is_lem_=p['is_lem'],is_stop_=p['is_stop'],is_stem_=p['is_stem'],is_alpha_=p['is_alpha'],tf_idf_=p['tf_idf'],alpha_ = p['alpha'],min_df_=p['min_df'], max_df_=p['max_df'],path='/content/drive/Shareddrives/Redes neuronales/TP1/data_results/param_preprocess/')
    print(score)
    scores.append(score) 
    print('done.')
preproc_hyperparams_70['score'] = scores

{'is_lc': False, 'is_lem': False, 'is_stop': False, 'is_stem': False, 'is_alpha': False, 'alpha': 1.0, 'min_df': 0.1, 'max_df': 0.9, 'tf_idf': True}
loading preprocessed text....
evaluating model....
0.39689
done.
{'is_lc': False, 'is_lem': True, 'is_stop': False, 'is_stem': True, 'is_alpha': False, 'alpha': 2.0, 'min_df': 1e-05, 'max_df': 0.8, 'tf_idf': False}
loading preprocessed text....
evaluating model....
0.6268466666666667
done.
{'is_lc': True, 'is_lem': True, 'is_stop': False, 'is_stem': True, 'is_alpha': True, 'alpha': 0.01, 'min_df': 1e-05, 'max_df': 0.8, 'tf_idf': False}
loading preprocessed text....
evaluating model....
0.62435
done.
{'is_lc': True, 'is_lem': False, 'is_stop': True, 'is_stem': False, 'is_alpha': False, 'alpha': 0.01, 'min_df': 0.1, 'max_df': 0.9, 'tf_idf': True}
loading preprocessed text....
evaluating model....
0.35018333333333335
done.
{'is_lc': False, 'is_lem': False, 'is_stop': True, 'is_stem': True, 'is_alpha': False, 'alpha': 1e-05, 'min_df': 2e-06, '

In [ ]:
preproc_hyperparams_70.sort_values('score',ascending=False)

###Tomamos las 10 combinaciones que nos dieron mejores resultados y las evaluamos con todo el conjunto:

In [ ]:
best_preproc_hyperparams = preproc_hyperparams_70.sort_values('score',ascending=False)[0:10]
best_preproc_hyperparams

In [ ]:
best_preproc_hyperparams = best_preproc_hyperparams.drop('score',axis=1)
best_preproc_hyperparams

In [ ]:
#tardo  min s
scores = []
for index, p in best_preproc_hyperparams.iterrows():
    print(p.to_dict())
    score = crossval_score_parameters_preprocessing(text_train_valid,labels_train_valid,is_lc_=p['is_lc'],is_lem_=p['is_lem'],is_stop_=p['is_stop'],is_stem_=p['is_stem'],is_alpha_=p['is_alpha'],tf_idf_=p['tf_idf'],alpha_ = p['alpha'],min_df_=p['min_df'], max_df_=p['max_df'],path_='/content/drive/Shareddrives/Redes neuronales/TP1/data_results_all_params/')
    print(score)
    scores.append(score) 
    print('done.')
best_preproc_hyperparams['score'] = scores

In [ ]:
best_preproc_hyperparams.sort_values('score',ascending=False)

# NBMN

##Hiperparámetros Naive Bayes

###Creación de hiperparámetros

In [ ]:
random_seed = 0
hyperparameters_specs = { 
    'alpha':   [1e-10,1e-5,0.01,1,2],#0.01, 0.1, 1.0,10.0 #smooothing laplaciano (para sumar cuando frec=0), más grande, más parecido al uniforme
    'min_df':  [0.1,0.0001,0.00001,0.000002], #0.01, 0.1, 0.05, 0.49, 1, 10 #porcentaje mínimo de aparición en documentos
    'max_df':  [0.4, 0.8,0.9], # 0.5, 0.7,#porcentaje máximo de aparición en documentos
    'tf_idf':  [True, False], #usar tfidf o countVectorizer
}

In [ ]:
# DEFINING HYPERPARAMETERS: hacemos las combinaciones
def get_hyperparameters():
    #for isalpha in hyperparameters_specs['isalpha']:
      for tf_idf in hyperparameters_specs['tf_idf']:
          for min_df in hyperparameters_specs['min_df']:
              for max_df in hyperparameters_specs['max_df']:
                  for alpha in hyperparameters_specs['alpha']:
                      hyperparameters = {
                          #'isalpha': isalpha,
                          'alpha':   alpha,
                          'min_df':  min_df,
                          'max_df':  max_df,
                          'tf_idf':  tf_idf,
                      }
                      
                      yield hyperparameters #gives a generator function, para evitar muchos llamados al código base.

###Obtenemos hipotesis filtradas

Se evaluan los hiperparámetros para  los conjuntos a los que se les hace:
- Lemming y stemming
- Lowcase, lemming y stemming	

In [ ]:
text_train_filtrado_lem_stem = open_txt('train_filtrado_lem_stem')
text_val_filtrado_lem_stem = open_txt('valid_filtrado_lem_stem')

In [ ]:
text_train_filtrado_lc_lem_stem = open_txt('train_filtrado_lc_lem_stem')
text_val_filtrado_lc_lem_stem = open_txt('valid_filtrado_lc_lem_stem')

### Evalúo los hiperparametros




#### Para las hipotesis con lemming y stemming

In [ ]:
hyperparameters_lem_stem = pd.DataFrame()

for hp in get_hyperparameters():
    df = pd.DataFrame(hp, index=[0])
    hyperparameters_lem_stem = hyperparameters_lem_stem.append(df,ignore_index=True) #ponemos combinaciones

hyperparameters_lem_stem 

,alpha,min_df,max_df,tf_idf
0,1.000000e-10,0.100000,0.4,True
1,1.000000e-05,0.100000,0.4,True
2,1.000000e-02,0.100000,0.4,True
3,1.000000e+00,0.100000,0.4,True
4,2.000000e+00,0.100000,0.4,True
...,...,...,...,...
115,1.000000e-10,0.000002,0.9,False
116,1.000000e-05,0.000002,0.9,False
117,1.000000e-02,0.000002,0.9,False
118,1.000000e+00,0.000002,0.9,False


In [ ]:
scores = []
for index, hp in hyperparameters_lem_stem.iterrows():
    print(hp.to_dict())
    score = run_sklearn(text_train_filtrado_lem_stem,labels_train,text_val_filtrado_lem_stem,labels_val,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters_lem_stem['score'] = scores 

{'alpha': 1e-10, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 1e-05, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 0.01, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 1.0, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 2.0, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 1e-10, 'min_df': 0.1, 'max_df': 0.8, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.40052834789676894
done.
{'alpha': 1e-05, 'min_df': 0.1, 'max_df': 0.8, 'tf_idf': True}
training model...
evaluating hyperparameters...
0.40052834789676894
done.
{'alpha': 0.01, 'min_df': 0.1, 'max_df': 0.8, 'tf_idf': True}

In [ ]:
hyperparameters_lem_stem

,alpha,min_df,max_df,tf_idf,score
0,1.000000e-10,0.100000,0.4,True,0.353180
1,1.000000e-05,0.100000,0.4,True,0.353180
2,1.000000e-02,0.100000,0.4,True,0.353180
3,1.000000e+00,0.100000,0.4,True,0.353180
4,2.000000e+00,0.100000,0.4,True,0.353180
...,...,...,...,...,...
115,1.000000e-10,0.000002,0.9,False,0.638996
116,1.000000e-05,0.000002,0.9,False,0.640012
117,1.000000e-02,0.000002,0.9,False,0.644686
118,1.000000e+00,0.000002,0.9,False,0.646312


In [ ]:
hyperparameters_lem_stem.sort_values('score',ascending=False)

,alpha,min_df,max_df,tf_idf,score
53,1.000000e+00,0.000002,0.8,True,0.656066
58,1.000000e+00,0.000002,0.9,True,0.656066
48,1.000000e+00,0.000002,0.4,True,0.655355
59,2.000000e+00,0.000002,0.9,True,0.653119
54,2.000000e+00,0.000002,0.8,True,0.653119
...,...,...,...,...,...
1,1.000000e-05,0.100000,0.4,True,0.353180
4,2.000000e+00,0.100000,0.4,True,0.353180
3,1.000000e+00,0.100000,0.4,True,0.353180
2,1.000000e-02,0.100000,0.4,True,0.353180


#### Para las hipotesis con lowcase, lemming y stemming

In [ ]:
hyperparameters = pd.DataFrame()

for hp in get_hyperparameters():
    df = pd.DataFrame(hp, index=[0])
    hyperparameters = hyperparameters.append(df,ignore_index=True) #ponemos combinaciones

hyperparameters 

,alpha,min_df,max_df,tf_idf
0,1.000000e-10,0.100000,0.4,True
1,1.000000e-05,0.100000,0.4,True
2,1.000000e-02,0.100000,0.4,True
3,1.000000e+00,0.100000,0.4,True
4,2.000000e+00,0.100000,0.4,True
...,...,...,...,...
115,1.000000e-10,0.000002,0.9,False
116,1.000000e-05,0.000002,0.9,False
117,1.000000e-02,0.000002,0.9,False
118,1.000000e+00,0.000002,0.9,False


In [ ]:
scores = []
for index, hp in hyperparameters.iterrows():
    print(hp.to_dict())
    score = run_sklearn(text_train_filtrado_lc_lem_stem,labels_train,text_val_filtrado_lc_lem_stem,labels_val,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters['score'] = scores 

{'alpha': 1e-10, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True, 'score': 0.35318024791709}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 1e-05, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True, 'score': 0.35318024791709}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 0.01, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True, 'score': 0.35318024791709}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 1.0, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True, 'score': 0.35318024791709}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 2.0, 'min_df': 0.1, 'max_df': 0.4, 'tf_idf': True, 'score': 0.35318024791709}
training model...
evaluating hyperparameters...
0.35318024791709
done.
{'alpha': 1e-10, 'min_df': 0.1, 'max_df': 0.8, 'tf_idf': True, 'score': 0.40052834789676894}
training model...
evaluating hyperparameters...
0.40042674253200566
done.
{'alpha': 1e-05, 'min_df': 0.1, '

In [ ]:
hyperparameters.sort_values('score',ascending=False)

,alpha,min_df,max_df,tf_idf,score
53,1.000000e+00,0.000002,0.8,True,0.656066
58,1.000000e+00,0.000002,0.9,True,0.656066
48,1.000000e+00,0.000002,0.4,True,0.655355
59,2.000000e+00,0.000002,0.9,True,0.653119
54,2.000000e+00,0.000002,0.8,True,0.653119
...,...,...,...,...,...
1,1.000000e-05,0.100000,0.4,True,0.353180
4,2.000000e+00,0.100000,0.4,True,0.353180
3,1.000000e+00,0.100000,0.4,True,0.353180
2,1.000000e-02,0.100000,0.4,True,0.353180


# MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

In [ ]:
cv = TfidfVectorizer(min_df=0.0000001, max_df = 0.9, ngram_range = (1,2)) #min_df: 0.00001,0.000001,0.0000001, ngram_range = (1,2)

In [ ]:
cv_train = cv.fit_transform(text_train_filtrados_stem)#
cv_valid = cv.transform(text_val_filtrados_stem) #con el test no see hace el fit

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=10).fit(cv_train, labels_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
clf.score(cv_valid,labels_valid)

###Solo haciendo low case

In [ ]:
score1 = NBMN_score(text_train_filtrado_lc,labels_train,text_val_filtrado_lc,labels_val)
score1

0.6543385490753911

In [ ]:
score1 = NBMN_score(text_train_filtrado_lc,labels_train,text_val_filtrado_lc,labels_val,ngram_range_ = (1,4))
score1

0.6546433651696809

In [ ]:
score1_1 = NBMN_score(text_train_filtrado_lc,labels_train,text_val_filtrado_lc,labels_val,ngram_range_ = (1,3))
score1_1

0.6525096525096525

### Low case, alpha, stop

In [ ]:
score1_2 = NBMN_score(text_train_filtrado_lc_stop_alpha,labels_train,text_val_filtrado_lc_stop_alpha,labels_val)
score1_2

0.6309693151798415


### Low case, alpha

In [ ]:
score1_2 = NBMN_score(text_train_filtrado_lc_alpha,labels_train,text_val_filtrado_lc_alpha,labels_val)
score1_2

0.6546433651696809

###Lemming sin stemming

In [ ]:
score2 = NBMN_score(text_train_filtrado_lc_lem_stop_alpha,labels_train,text_val_filtrado_lc_lem_stop_alpha,labels_val)
score2

0.6323917902865271

###Stemming sin Lemming

In [ ]:
score3 = NBMN_score(text_train_filtrado_lc_stop_stem_alpha,labels_train,text_val_filtrado_lc_stop_stem_alpha,labels_val)
score3

0.6338142653932127

###Con todo

In [ ]:
score4 = NBMN_score(text_train_filtrado_lc_lem_stop_stem_alpha,labels_train,text_val_filtrado_lc_lem_stop_stem_alpha,labels_val)
score4

0.6352367404998984

###Con todo menos stop words

In [ ]:
score5 = NBMN_score(text_train_filtrado_lc_lem_stem_alpha,labels_train,text_val_filtrado_lc_lem_stem_alpha,labels_val)
score5

0.6556594188173136

# Ejemplos

In [ ]:
#cv = CountVectorizer(min_df=10, max_df = 0.8, ngram_range = (1,2)) 
#min_df es que las palabras aparezcan por lo menos en 10 documentos
cv = TfidfVectorizer(min_df=0.0000001, max_df = 0.9, ngram_range = (1,2)) #min_df: 0.00001,0.000001,0.0000001, ngram_range = (1,2)
#ngram_range genera tokens con palabras tomadas de a 1 y palabras tomadas de a 2
#Empieza a tomar las palabras de a 2 y las toma como tokens
#ya empiezo a calcular algo de la estructura de la oración
#Toma palabras de a dos como si fueran un vocabulrio, 
#Fundamental mirar el min_df cuando lo hagamos

In [ ]:
cv_train = cv.fit_transform(text_train_filtrados_stem)#
clf = MultinomialNB(alpha=2) #1e-10
clf.fit(cv_train, df_train['gold_label'])

MultinomialNB(alpha=2, class_prior=None, fit_prior=True)

In [ ]:
cv_valid = cv.transform(text_val_filtrados_stem) #con el test no see hace el fit

In [ ]:
# Veamos còomo funciona el clasificador para valid
clf.score(cv_valid, labels_val)#NOS INTERESA EL DE VALIDACION
#nos da el accuracy

0.6335094492989229

En lugar del count vectorizer podemos probar el Tfidf:

In [ ]:
cv2 = TfidfVectorizer(min_df=1)

In [ ]:
cv2_train = cv2.fit_transform(text_train)

In [ ]:
clf.score(cv_train, df_train['gold_label'])

0.6525692296770647

In [ ]:
cv2_valid = cv2.transform(text_val)

In [ ]:
cv_train.shape #(cantidad de docs, cant palabras vocabulario)

In [ ]:
cv2_train.shape

In [ ]:
cv2.get_feature_names()[-19:]

In [ ]:
# Vemos un poco el vocabulario
cv.get_feature_names()[-19:]

In [ ]:
#para armar modelo Bayes, el multinomial:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB(alpha=1e-10) #1e-10
clf.fit(cv_train, labels_train) #text_train_filtrados

MultinomialNB(alpha=1e-10, class_prior=None, fit_prior=True)

In [ ]:
clf = MultinomialNB(alpha=0.01)
clf.fit(cv_train, df_train['gold_label'])

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [ ]:
clf2 = MultinomialNB(alpha=1e-10)
#puedo mejorar el modelo cambiando el alpha
clf2.fit(cv2_train, labels_train)

NameError: ignored

In [ ]:
#logprobabilidades de la clase 0
clf.coef_[0]
#son coeficientes negativos porque son las log(probabilidades)

array([-12.54799145, -37.18328029, -14.15742936, ..., -14.15742936,
       -14.15742936, -14.15742936])

In [ ]:
clf.coef_.shape
#como tenemos 3 clases, tenemos 3 resultados posibles

In [ ]:
#Veamos còmo funciona el clasificador para train
clf.score(cv_train, labels_train)

0.7892392517206166

In [ ]:
cv_valid

<9842x51868 sparse matrix of type '<class 'numpy.int64'>'
	with 54708 stored elements in Compressed Sparse Row format>

In [ ]:
clf2.score(cv2_valid, labels_val)

El dataset parace estar bastante sesgado porque ya la primera vez que probamos tenemos un accuracy del 60\%

Podemos ir tocando el alpha y el min_df para ver como va cambiando, pensamos que puede generalizar mejor el modelo cuando subo alpha porque estoy diciendo que las probabilidades que valen cero, les sumo un alpha, estoy diciendo que lo que vimos de los datos, nos quedemos cada vez con menos información.

In [ ]:
cv_test = cv.transform(text_test) #count vectorizer de test
test_labels = clf.predict(cv_test)

In [ ]:
#Armo el submission.csv
df_test = pd.DataFrame(data=test_labels, columns=["pred_labels"],)

In [ ]:
df_test.head()

In [ ]:
df_test.index.names = ["pairID"]

In [ ]:
df_test

In [ ]:
df_test.to_csv("submission.csv")

Se espera de nuestras notebooks que tengamos un submission.csv de salida, si ponemos save notebook mete el submission como respuesta.

El score que importa es el sumbmission de test, como tenes que separar en test y validation, el de validation va a ser el que más se parece a test porque no fue usado para obtener los parámetros. Entonces, una vez que optimizas todo para test, lo haces para validación y lo subis. 

Tenemos que ver que hacer de preprocesamiento para que este modelo mejore.

#Ejemplos hiperparams

####Para los textos sin stemming

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_train_filtrados_sinStem.txt', 'rb') as fp:
    text_train_filtrados_sinStem = pickle.load(fp)

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_val_filtrados_sinStem.txt', 'rb') as fp:
    text_val_filtrados_sinStem = pickle.load(fp)

In [ ]:
# PREPROCESSING DATASET
print('preprocessing dataset...')
scores = []
for index, hp in hyperparameters.iterrows():
    print(hp.to_dict())
    score = run_sklearn(text_train_filtrados_sinStem,labels_train,text_val_filtrados_sinStem,labels_val,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters['score'] = scores    

In [ ]:
hyperparameters.sort_values(by=['score'],ascending=False)[0:15]

,alpha,min_df,max_df,tf_idf,score
69,2.0,0.000001,0.8,True,0.632697
74,2.0,0.000001,0.9,True,0.632697
64,2.0,0.000001,0.7,True,0.632697
54,2.0,0.000001,0.4,True,0.632697
59,2.0,0.000001,0.6,True,0.632697
58,1.0,0.000001,0.6,True,0.632392
68,1.0,0.000001,0.8,True,0.632392
53,1.0,0.000001,0.4,True,0.632392
63,1.0,0.000001,0.7,True,0.632392
73,1.0,0.000001,0.9,True,0.632392


#### Para los textos con stemming

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_train_filtrados_stem.txt', 'rb') as fp:
    text_train_filtrados_stem = pickle.load(fp)

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_val_filtrados_stem.txt', 'rb') as fp:
    text_val_filtrados_stem = pickle.load(fp)

In [ ]:
# PREPROCESSING DATASET
print('preprocessing dataset...')
scores = []
for index, hp in hyperparameters.iterrows():
    print(hp.to_dict())
    score = run_sklearn(text_train_filtrados_stem,labels_train,text_val_filtrados_stem,labels_val,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters['score'] = scores    

In [ ]:
hyperparameters.sort_values(by=['score'],ascending=False)[0:15]

,alpha,min_df,max_df,tf_idf,score
68,1.0,0.000001,0.8,True,0.635237
53,1.0,0.000001,0.4,True,0.635237
63,1.0,0.000001,0.7,True,0.635237
73,1.0,0.000001,0.9,True,0.635237
58,1.0,0.000001,0.6,True,0.635237
69,2.0,0.000001,0.8,True,0.633916
64,2.0,0.000001,0.7,True,0.633916
74,2.0,0.000001,0.9,True,0.633916
59,2.0,0.000001,0.6,True,0.633916
54,2.0,0.000001,0.4,True,0.633916


#### Para los textos con solo stemming

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_train_filtrados_soloStem.txt', 'rb') as fp:
    text_train_filtrados_soloStem = pickle.load(fp)

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_val_filtrados_soloStem.txt', 'rb') as fp:
    text_val_filtrados_soloStem = pickle.load(fp)

In [ ]:
# PREPROCESSING DATASET
print('preprocessing dataset...')
scores = []
for index, hp in hyperparameters.iterrows():
    print(hp.to_dict())
    score = run_sklearn(text_train_filtrados_soloStem,labels_train,text_val_filtrados_soloStem,labels_val,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters['score'] = scores    

preprocessing dataset...
{'alpha': 1e-10, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.6004877057508636}
training model...
evaluating hyperparameters...
0.5982523877260719
done.
{'alpha': 1e-05, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.6004877057508636}
training model...
evaluating hyperparameters...
0.5982523877260719
done.
{'alpha': 0.01, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.6004877057508636}
training model...
evaluating hyperparameters...
0.5982523877260719
done.
{'alpha': 1.0, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.6002844950213371}
training model...
evaluating hyperparameters...
0.5982523877260719
done.
{'alpha': 2.0, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.6003861003861004}
training model...
evaluating hyperparameters...
0.599065230644178
done.
{'alpha': 1e-10, 'min_df': 0.0001, 'max_df': 0.6, 'tf_idf': True, 'score': 0.6004877057508636}
training model...
evaluating hyperparameters..

In [ ]:
hyperparameters.sort_values(by=['score'],ascending=False)[0:15]

,alpha,min_df,max_df,tf_idf,score
53,1.0,0.000001,0.4,True,0.632798
73,1.0,0.000001,0.9,True,0.632798
58,1.0,0.000001,0.6,True,0.632798
68,1.0,0.000001,0.8,True,0.632798
63,1.0,0.000001,0.7,True,0.632798
64,2.0,0.000001,0.7,True,0.630461
74,2.0,0.000001,0.9,True,0.630461
54,2.0,0.000001,0.4,True,0.630461
69,2.0,0.000001,0.8,True,0.630461
59,2.0,0.000001,0.6,True,0.630461


#### Para los textos con todo

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_train_filtrados_todo.txt', 'rb') as fp:
    text_train_filtrados_todo = pickle.load(fp)

In [ ]:
with open ('/content/drive/Shareddrives/Redes neuronales/TP1/data/text_val_filtrados_todo.txt', 'rb') as fp:
    text_val_filtrados_todo = pickle.load(fp)

In [ ]:
# PREPROCESSING DATASET
print('preprocessing dataset...')
scores = []
for index, hp in hyperparameters.iterrows():
    print(hp.to_dict())
    score = run_sklearn(text_train_filtrados_todo,labels_train,text_val_filtrados_todo,labels_val,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters['score'] = scores    

preprocessing dataset...
{'alpha': 1e-10, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.5982523877260719}
training model...
evaluating hyperparameters...
0.595915464336517
done.
{'alpha': 1e-05, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.5982523877260719}
training model...
evaluating hyperparameters...
0.595915464336517
done.
{'alpha': 0.01, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.5982523877260719}
training model...
evaluating hyperparameters...
0.595915464336517
done.
{'alpha': 1.0, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.5982523877260719}
training model...
evaluating hyperparameters...
0.5961186750660434
done.
{'alpha': 2.0, 'min_df': 0.0001, 'max_df': 0.4, 'tf_idf': True, 'score': 0.599065230644178}
training model...
evaluating hyperparameters...
0.5965250965250966
done.
{'alpha': 1e-10, 'min_df': 0.0001, 'max_df': 0.6, 'tf_idf': True, 'score': 0.5982523877260719}
training model...
evaluating hyperparameters...
0

In [ ]:
hyperparameters.sort_values(by=['score'],ascending=False)[0:15]

,alpha,min_df,max_df,tf_idf,score
68,1.0,0.000001,0.8,True,0.630664
73,1.0,0.000001,0.9,True,0.630664
63,1.0,0.000001,0.7,True,0.630664
58,1.0,0.000001,0.6,True,0.630664
53,1.0,0.000001,0.4,True,0.630664
74,2.0,0.000001,0.9,True,0.629344
64,2.0,0.000001,0.7,True,0.629344
59,2.0,0.000001,0.6,True,0.629344
54,2.0,0.000001,0.4,True,0.629344
69,2.0,0.000001,0.8,True,0.629344
